# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 15 2022 12:00AM,239597,10,Enova-10138,Emerginnova,Released
1,Jun 15 2022 12:00AM,239596,10,SNAP4515177999,"Snap Medical Industries, LLC",Released
2,Jun 15 2022 12:00AM,239596,10,SNAP614221,"Snap Medical Industries, LLC",Released
3,Jun 15 2022 12:00AM,239592,10,8144802,Yusen – 3D Matrix,Released
4,Jun 15 2022 12:00AM,239594,10,8144807,Yusen – 3D Matrix,Released
5,Jun 14 2022 3:47PM,239595,21,617356,"NBTY Global, Inc.",Released
6,Jun 14 2022 3:45PM,239593,21,617359,"NBTY Global, Inc.",Released
7,Jun 14 2022 3:30PM,239591,19,ADV80003316,"AdvaGen Pharma, Ltd",Released
8,Jun 14 2022 3:30PM,239591,19,ADV80003317,"AdvaGen Pharma, Ltd",Released
9,Jun 14 2022 3:30PM,239591,19,ADV80003318,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,239593,Released,1
47,239594,Released,1
48,239595,Released,1
49,239596,Released,2
50,239597,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239593,NaN,NaN,1.0
239594,NaN,NaN,1.0
239595,NaN,NaN,1.0
239596,NaN,NaN,2.0
239597,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,36.0,27.0,90.0
239523,0.0,0.0,5.0
239525,0.0,0.0,1.0
239526,3.0,3.0,8.0
239527,0.0,0.0,52.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,36,27,90
239523,0,0,5
239525,0,0,1
239526,3,3,8
239527,0,0,52


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,36,27,90
1,239523,0,0,5
2,239525,0,0,1
3,239526,3,3,8
4,239527,0,0,52


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,36,27,90
1,239523,,,5
2,239525,,,1
3,239526,3,3,8
4,239527,,,52


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 15 2022 12:00AM,239597,10,Emerginnova
1,Jun 15 2022 12:00AM,239596,10,"Snap Medical Industries, LLC"
3,Jun 15 2022 12:00AM,239592,10,Yusen – 3D Matrix
4,Jun 15 2022 12:00AM,239594,10,Yusen – 3D Matrix
5,Jun 14 2022 3:47PM,239595,21,"NBTY Global, Inc."
6,Jun 14 2022 3:45PM,239593,21,"NBTY Global, Inc."
7,Jun 14 2022 3:30PM,239591,19,"AdvaGen Pharma, Ltd"
34,Jun 14 2022 3:27PM,239590,19,"GUSA Granules USA, Inc."
35,Jun 14 2022 3:21PM,239589,21,"NBTY Global, Inc."
36,Jun 14 2022 2:35PM,239587,18,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 15 2022 12:00AM,239597,10,Emerginnova,,,1
1,Jun 15 2022 12:00AM,239596,10,"Snap Medical Industries, LLC",,,2
2,Jun 15 2022 12:00AM,239592,10,Yusen – 3D Matrix,,,1
3,Jun 15 2022 12:00AM,239594,10,Yusen – 3D Matrix,,,1
4,Jun 14 2022 3:47PM,239595,21,"NBTY Global, Inc.",,,1
5,Jun 14 2022 3:45PM,239593,21,"NBTY Global, Inc.",,,1
6,Jun 14 2022 3:30PM,239591,19,"AdvaGen Pharma, Ltd",,,27
7,Jun 14 2022 3:27PM,239590,19,"GUSA Granules USA, Inc.",,,1
8,Jun 14 2022 3:21PM,239589,21,"NBTY Global, Inc.",,,1
9,Jun 14 2022 2:35PM,239587,18,"ClearSpec, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 12:00AM,239597,10,Emerginnova,1,,
1,Jun 15 2022 12:00AM,239596,10,"Snap Medical Industries, LLC",2,,
2,Jun 15 2022 12:00AM,239592,10,Yusen – 3D Matrix,1,,
3,Jun 15 2022 12:00AM,239594,10,Yusen – 3D Matrix,1,,
4,Jun 14 2022 3:47PM,239595,21,"NBTY Global, Inc.",1,,
5,Jun 14 2022 3:45PM,239593,21,"NBTY Global, Inc.",1,,
6,Jun 14 2022 3:30PM,239591,19,"AdvaGen Pharma, Ltd",27,,
7,Jun 14 2022 3:27PM,239590,19,"GUSA Granules USA, Inc.",1,,
8,Jun 14 2022 3:21PM,239589,21,"NBTY Global, Inc.",1,,
9,Jun 14 2022 2:35PM,239587,18,"ClearSpec, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 12:00AM,239597,10,Emerginnova,1,,
1,Jun 15 2022 12:00AM,239596,10,"Snap Medical Industries, LLC",2,,
2,Jun 15 2022 12:00AM,239592,10,Yusen – 3D Matrix,1,,
3,Jun 15 2022 12:00AM,239594,10,Yusen – 3D Matrix,1,,
4,Jun 14 2022 3:47PM,239595,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 12:00AM,239597,10,Emerginnova,1.0,NaN,NaN
1,Jun 15 2022 12:00AM,239596,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
2,Jun 15 2022 12:00AM,239592,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Jun 15 2022 12:00AM,239594,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Jun 14 2022 3:47PM,239595,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 12:00AM,239597,10,Emerginnova,1.0,0.0,0.0
1,Jun 15 2022 12:00AM,239596,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
2,Jun 15 2022 12:00AM,239592,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Jun 15 2022 12:00AM,239594,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Jun 14 2022 3:47PM,239595,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4072495,114.0,24.0,15.0
12,239544,0.0,2.0,0.0
15,478984,94.0,41.0,44.0
18,1197874,5.0,0.0,0.0
19,1916604,38.0,0.0,0.0
20,479086,19.0,28.0,11.0
21,958361,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4072495,114.0,24.0,15.0
1,12,239544,0.0,2.0,0.0
2,15,478984,94.0,41.0,44.0
3,18,1197874,5.0,0.0,0.0
4,19,1916604,38.0,0.0,0.0
5,20,479086,19.0,28.0,11.0
6,21,958361,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,114.0,24.0,15.0
1,12,0.0,2.0,0.0
2,15,94.0,41.0,44.0
3,18,5.0,0.0,0.0
4,19,38.0,0.0,0.0
5,20,19.0,28.0,11.0
6,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,114.0
1,12,Released,0.0
2,15,Released,94.0
3,18,Released,5.0
4,19,Released,38.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21
Status,,,,,,,
Completed,15.0,0.0,44.0,0.0,0.0,11.0,0.0
Executing,24.0,2.0,41.0,0.0,0.0,28.0,1.0
Released,114.0,0.0,94.0,5.0,38.0,19.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21
0,Completed,15.0,0.0,44.0,0.0,0.0,11.0,0.0
1,Executing,24.0,2.0,41.0,0.0,0.0,28.0,1.0
2,Released,114.0,0.0,94.0,5.0,38.0,19.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21
0,Completed,15.0,0.0,44.0,0.0,0.0,11.0,0.0
1,Executing,24.0,2.0,41.0,0.0,0.0,28.0,1.0
2,Released,114.0,0.0,94.0,5.0,38.0,19.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()